In [111]:
import pandas as pd
import numpy as np
import httplib2
from bs4 import BeautifulSoup
import requests
from io import StringIO
import datetime

In [112]:
# Выбор программы обучения
program = ['management','log', 'gmu', 'economics']
url = 'https://spb.hse.ru/ba/'+ program[0] + '/'

In [113]:
# Выбор курса и модуля (надо прикрутить учебный год который year=2020)
now = datetime.datetime.now()
if int(now.month)<=7:
  year_edu = now.year-1
else:
  year_edu = now.year

course_study = ['1','2','3','4']
module = ['1','2','3','4']

url = url + 'courses?course=' + course_study[3]+ '.' + module[0] + '.' + course_study[3] + '.' + module[3]+'&page=1&year='+str(year_edu)

In [114]:
# проверка сервера и получение кода всей страницы для дальнейших манипуляций
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [115]:
# Парсинг всех страниц
pages = soup.find_all(class_='pages__page')
pages_links = []
for i in range(1, len(pages)):
    pages_links.append(str(soup.find_all(class_='pages__page')[i]).replace('<a class="pages__page" href=','').split(">")[0].replace('"',''))

In [116]:
# Парсинг названий курсов и ссылки на них
courses_names=[]
courses_links=[]
courses = soup.find_all(class_='link link_dark')
for i in range(len(courses)):
    courses_names.append(str(soup.find_all(class_='link link_dark')[i].getText()))
    courses_links.append(str(soup.find_all(class_='link link_dark')[i]).replace('<a class="link link_dark" href=','>'))
    if (i == (len(courses)-1)) and (len(pages_links)>0):
        for j in range(len(pages_links)):
            page = requests.get(pages_links[j])
            soup = BeautifulSoup(page.text, 'lxml')
            courses = soup.find_all(class_='link link_dark')
            for k in range(len(courses)):
                        courses_names.append(str(soup.find_all(class_='link link_dark')[k].getText()))
                        courses_links.append(str(soup.find_all(class_='link link_dark')[k]).replace('<a class="link link_dark" href=','>'))
for i in range(len(courses_links)):
  courses_links[i]=courses_links[i].split(">")[1].replace('"','')

In [119]:
# Получаем страницу курса и информацию о нем
url = courses_links[5]
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
url

'https://spb.hse.ru/ba/management/courses/376727214.html'

In [126]:
# Получаем формулу оценки, преобразуем и разделяяем коэффициенты
# c https://spb.hse.ru/ba/management/courses/375310631.html не работает
checkon = soup.find_all(class_='grey i')
if len(checkon)<1:
  print('Sorry, no information about the grades :(')
else:
  formula = str(soup.find_all(class_='grey i')[-1].getText())
  StringData = StringIO(formula)
  grades = pd.read_csv(StringData, sep ="+", header=None)
  numb=0
  activity=[]
  for col in grades:
     activity.append(str(grades.iloc[0][numb].split('*')[-1]))
     grades.iloc[0][numb]= (grades.iloc[0][numb].split('*')[0])
     numb+=1
  grades.columns=activity
  grades.rename(index={0:'Coefficients'}, inplace=True)
grades
grades.to_excel('/testing.xlsx')